In [3]:
import sys
import os

# Esse comando procura a pasta onde o pip instalou os pacotes do usuário
user_site = os.path.join(os.environ['APPDATA'], 'Python', 'Python313', 'site-packages')

# Adiciona esse caminho na memória do Jupyter agora
if user_site not in sys.path:
    sys.path.append(user_site)

# Teste de importação
try:
    import pdfplumber
    print("✅ SUCESSO! O Jupyter agora consegue ler o pdfplumber.")
except ImportError:
    print("❌ Ainda não encontrou. Vamos tentar o próximo passo.")

✅ SUCESSO! O Jupyter agora consegue ler o pdfplumber.


In [4]:
!pip install pdfplumber openpyxl pandas
import pandas as pd
import os
import pdfplumber
import re
import sys
!{sys.executable} -m pip install pdfplumber --user

Defaulting to user installation because normal site-packages is not writeable


In [14]:


from IPython.display import FileLink, display
# --- 1. CONFIGURAÇÃO DE AMBIENTE ---
# Garante que o Python encontre as bibliotecas no caminho do Anaconda/Usuário
caminho_lib = r'C:\Users\Kaliton Oliveira\AppData\Roaming\Python\Python313\site-packages'
if caminho_lib not in sys.path: 
    sys.path.append(caminho_lib)

try:
    import pdfplumber
except ImportError:
    print("⚠️ PDFPlumber não encontrado. Verifique a instalação para processar PDFs.")

# --- 2. CONFIGURAÇÕES DO PROJETO ---
PASTA_ENTRADA = './DADOS_BRUTOS'
ARQUIVO_FINAL = 'Relatorio_Financeiro_Consolidado.xlsx'

# Lista de categorias oficiais solicitadas pelo cliente para conferência
CATEGORIAS_OFICIAIS = [
    'Transferência', 'Não identificado', 'Recebimento', 'Alimentação', 'Impostos',
    'Habitação', 'Saúde', 'Educação', 'Transporte', 'Lazer', 'Salário', 'Diarista',
    'Estacionamento', 'Ajuda de custo Fátima', 'Luz', 'Celular', 'Gás', 'Internet',
    'Assinaturas', 'Supermercado', 'Padaria', 'Farmácia', 'Cabeleireiro', 'Academia',
    'Manicure', 'Juros', 'Anuidade', 'Empréstimos', 'Pet'
]

def limpar_valor(valor):
    """ 
    Tratamento de Moeda: Converte 'R$ 1.500,80' (BR) em 1500.80 (Float)
    independente de como o dado venha sujo do PDF ou Excel.
    """
    if pd.isna(valor): return 0.0
    s = str(valor).upper().replace('R$', '').strip()
    # Lógica para detectar e corrigir separadores brasileiros
    if ',' in s and ('.' not in s or s.find('.') < s.find(',')):
        s = s.replace('.', '').replace(',', '.')
    else:
        s = s.replace(',', '')
    try:
        return float(s)
    except:
        return 0.0

def iniciar_automacao():
    print("🚀 Iniciando processamento dos arquivos (Modo Multi-Páginas)...")
    base_consolidada = []
    arquivos_nao_lidos = []

    # Cria a pasta de entrada caso ela não exista
    if not os.path.exists(PASTA_ENTRADA):
        os.makedirs(PASTA_ENTRADA)
        print(f"📁 Pasta {PASTA_ENTRADA} criada. Arraste os arquivos para lá.")
        return

    # --- 3. LOOP PRINCIPAL DE VARREDURA ---
    for arquivo in os.listdir(PASTA_ENTRADA):
        # Ignora arquivos temporários do sistema ou o próprio resultado
        if arquivo.startswith('~') or arquivo == ARQUIVO_FINAL: continue
        
        caminho = os.path.join(PASTA_ENTRADA, arquivo)
        df = pd.DataFrame()

        try:
            # --- LEITURA DE EXCEL ---
            if arquivo.endswith(('.xlsx', '.xls')):
                df = pd.read_excel(caminho)
            
            # --- LEITURA DE CSV ---
            elif arquivo.endswith('.csv'):
                df = pd.read_csv(caminho, sep=None, engine='python')
            
            # --- LEITURA DE PDF (NOVA LÓGICA MULTI-PÁGINAS) ---
            elif arquivo.endswith('.pdf'):
                with pdfplumber.open(caminho) as pdf:
                    paginas_pdf = []
                    for i, pagina in enumerate(pdf.pages):
                        tabela = pagina.extract_table()
                        if tabela:
                            # Converte tabela da página atual; usa 1ª linha como cabeçalho
                            df_pg = pd.DataFrame(tabela[1:], columns=tabela[0])
                            paginas_pdf.append(df_pg)
                    if paginas_pdf:
                        df = pd.concat(paginas_pdf, ignore_index=True)
            
            # --- 4. PADRONIZAÇÃO E LIMPEZA DOS DADOS ---
            if not df.empty:
                # Normaliza nomes de colunas para maiúsculas e remove espaços
                df.columns = [str(c).upper().strip() for c in df.columns]
                
                # Mapeia colunas variadas para o padrão exigido pelo cliente
                traducao = {'PRECO': 'VALOR', 'STIKER': 'DESCRITIVO', 'OPERAÇÃO': 'CATEGORIA'}
                df.rename(columns=traducao, inplace=True)
                
                # Adiciona a coluna FONTE (nome do arquivo original)
                df['FONTE'] = arquivo
                
                # Limpa e converte a coluna VALOR para formato numérico
                if 'VALOR' in df.columns:
                    df['VALOR'] = df['VALOR'].apply(limpar_valor)
                
                # Cria colunas de controle de parcelas (requisito do cliente)
                for col in ['HÁ PARCELAS - SIM OU NÃO', 'SE SIM, Nº TOTAL DE PARCELAS', 
                            'SE SIM, Nº DE PARCELAS PAGAS', 'SE SIM, Nº DE PARCELAS FALTANTES']:
                    if col not in df.columns:
                        df[col] = "NÃO" if "HÁ" in col else 0
                
                base_consolidada.append(df)
                print(f"✔️ Processado: {arquivo} ({len(df)} linhas encontradas)")

        except Exception as e:
            arquivos_nao_lidos.append(f"{arquivo} (Erro: {str(e)})")

    # --- 5. CONSOLIDAÇÃO FINAL E FORMATAÇÃO ---
    if base_consolidada:
        resultado = pd.concat(base_consolidada, ignore_index=True)
        
        # Define a ordem exata das colunas conforme o contrato
        ordem_final = ['DATA', 'VALOR', 'DESCRITIVO', 'FONTE', 'CATEGORIA', 
                       'HÁ PARCELAS - SIM OU NÃO', 'SE SIM, Nº TOTAL DE PARCELAS', 
                       'SE SIM, Nº DE PARCELAS PAGAS', 'SE SIM, Nº DE PARCELAS FALTANTES']
        resultado = resultado.reindex(columns=ordem_final)

        # Salva em Excel aplicando a Máscara Contábil Americana (1,000.00)
        with pd.ExcelWriter(ARQUIVO_FINAL, engine='openpyxl') as writer:
            resultado.to_excel(writer, index=False, sheet_name='Financeiro_Consolidado')
            ws = writer.sheets['Financeiro_Consolidado']
            
            # Aplica o formato #,##0.00 na coluna B (VALOR)
            for row in range(2, len(resultado) + 2):
                ws.cell(row=row, column=2).number_format = '#,##0.00'

        print(f"\n✅ SUCESSO! Relatório '{ARQUIVO_FINAL}' gerado.")
        # Gera o link azul para o cliente baixar
        display(FileLink(ARQUIVO_FINAL, result_html_prefix="📥 Clique para baixar o arquivo pronto: "))
    
    # Lista arquivos que falharam (Ex: Screenshots ou PDFs corrompidos)
    if arquivos_nao_lidos:
        print("\n⚠️ ARQUIVOS QUE EXIGEM CONFERÊNCIA MANUAL:")
        for item in arquivos_nao_lidos: 
            print(f" - {item}")

# Executa a automação
if __name__ == "__main__":
    iniciar_automacao()

🚀 Iniciando processamento dos arquivos (Modo Multi-Páginas)...
✔️ Processado: analise_mercado_acoes tarefa enviada dia 19_03_25 - mercado_acoes (1).csv (2000 linhas encontradas)
✔️ Processado: exemplo_cliente.xlsx (2 linhas encontradas)
✔️ Processado: extrato_teste.pdf (3 linhas encontradas)

✅ SUCESSO! Relatório 'Relatorio_Financeiro_Consolidado.xlsx' gerado.


C:\Users\Kaliton Oliveira\Desktop\Relatorio_Financeiro_Consolidado.xlsx